Поиск ассоциативных правил при помощи apriori алгоритма

Импортируем библиотеки

In [20]:
import numpy as np
import pandas as pd
from apyori import apriori

Считываем данные

In [ ]:
df = pd.read_excel('datasets\Big_retail_dataset.xlsx')
print("Количество строк: %d"  %  df.shape[0])
print("Количество столбцов: %d"  %  df.shape[1])

In [ ]:
df.head()

Нормализуем дату(избавляемся от времени)

In [ ]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate']).dt.normalize()
df.head()

Выводим количество строк с пустым атрибутом 'Description'

In [ ]:
print(len(df[df['Description'].isnull()]))

Удаляем строки с пустым атрибутом 'Description'

In [ ]:
df=df[df['Description'].notnull()]
print("Количество строк: %d"  %  df.shape[0])
print("Количество столбцов: %d"  %  df.shape[1])

In [ ]:
df.head()

Группируем данные по номерам выставленных счетов

In [ ]:
gp_InvoiceNo = df.groupby('InvoiceNo')

Формируем список транзакций, где каждая запись - это список вещей, купленных на один счет

In [ ]:
transactions = []
for name,group in gp_InvoiceNo:
    transactions.append(list(group['Description'].map(str)))

Формируем из DataFrame списка транзакций

In [ ]:
transactions_df=pd.DataFrame(transactions)
print("Количество строк: %d"  %  transactions_df.shape[0])
print("Количество столбцов: %d"  %  transactions_df.shape[1])

In [ ]:
transactions_df.head()

Вычисляем поддержку, при которой товар встречается в среднем один раз за день

In [ ]:
support=len(df['InvoiceDate'].unique())/transactions_df.shape[0]
print(support)

Используя функцию apriori, находим список ассоциативных правил

In [ ]:
association_results = list(apriori(transactions,
                                   min_support = support,
                                   min_confidence = 0.5,
                                   min_lift = 3,
                                   min_length = 2))

Формируем DataFrame для списка правил

In [ ]:
association_results_table=\
    pd.DataFrame(np.random.randint(low=0,
                                   high=1,
                                   size=(len(association_results),6)),
                 columns=['GeneralRules',
                          'LeftPart',
                          'RightPart',
                          'Support',
                          'Confidence',
                          'Lift'])



Заполняем DataFrame данными

In [ ]:
index=0
for g, s, i in association_results:
    association_results_table.iloc[index] = [' | & | '.join(list(g)),
                                             ' | & | '.join(list(i[0][0])),
                                             ' | & | '.join(list(i[0][1])),
                                             s,
                                             i[0][2],
                                             i[0][3]]
    index=index+1

Сортируем данные по атрибуту 'Lift'

In [ ]:
association_results_table = \
    association_results_table.sort_values('Lift', ascending=0)

Выводим правила в виде таблицы

In [ ]:
print("Количество строк: %d"  %  association_results_table.shape[0])
print("Количество столбцов: %d"  %  association_results_table.shape[1])

In [ ]:
association_results_table.head()

Выводим топ-10 в виде списка

In [ ]:
count=1
for i, d in association_results_table.head(10).iterrows():
    print('Rule #'+str(count)+':')
    print(d['LeftPart'])
    print('=> '+d['RightPart'])
    print('Support: '+str(d['Support']))
    print('Confidence: '+str(d['Confidence']))
    print('Lift: '+str(d['Lift']))
    print('==========================================================')
    count=count+1

Rule #1:
REGENCY TEA PLATE GREEN 
=> REGENCY TEA PLATE ROSES 
Support: 0.01317188906160517
Confidence: 0.8341968911917097
Lift: 44.62314486230314
Rule #2:
POPPY'S PLAYHOUSE BEDROOM 
=> POPPY'S PLAYHOUSE KITCHEN
Support: 0.01284463715945349
Confidence: 0.7370892018779343
Lift: 40.95200597524541
Rule #3:
SET/6 RED SPOTTY PAPER CUPS
=> SET/6 RED SPOTTY PAPER PLATES
Support: 0.014317270719136055
Confidence: 0.8177570093457943
Lift: 37.93337353029846
Rule #4:
SMALL DOLLY MIX DESIGN ORANGE BOWL
=> SMALL MARSHMALLOWS PINK BOWL
Support: 0.014317270719136055
Confidence: 0.6628787878787878
Lift: 36.17128314393939
Rule #5:
WOODEN HEART CHRISTMAS SCANDINAVIAN
=> WOODEN STAR CHRISTMAS SCANDINAVIAN
Support: 0.0159126237421255
Confidence: 0.7203703703703703
Lift: 34.19451276519237
Rule #6:
SET OF 12 FAIRY CAKE BAKING CASES
=> SET OF 12 MINI LOAF BAKING CASES
Support: 0.012558291745070768
Confidence: 0.5511669658886894
Lift: 30.0085248287637
Rule #7:
JUMBO BAG VINTAGE CHRISTMAS 
=> JUMBO BAG 50'S CHRI